In [10]:
import pandas as pd
import csv
from collections import defaultdict
import os


def get_test_passage_query_pairs():
    file_path = "splitted_data.txt"

    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        df = pd.DataFrame(reader, columns=['turn_id', 'user_utterance', 'response',  'passage_id', 'passage_txt', 'score', 'ptkb', 'lable'])
    df.head()   

    print(set(list(df['lable'])))
    test_set = df[df['lable']=='test']
    print(len(test_set))
    turn_passages_test = defaultdict(list)

    for _, row in test_set.iterrows():
        turn_passages_test[row["turn_id"]].append(row["passage_id"])
    
    return turn_passages_test

def create_15_percent_pool(new_pool_path, nist_pool_path):
    print(new_pool_path)
    turn_passages_test = get_test_passage_query_pairs()
    new_lines = []

    with open(new_pool_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        turn_id, _, passage_id, score = line.split('\t')
        if passage_id in turn_passages_test[turn_id]:
            new_lines.append(line)
    print(len(new_lines))
    assert len(new_lines) == 917

    # with open(nist_pool_path, 'r') as f:
    #     lines = f.readlines()

    # for line in lines:
    #     turn_id, _, passage_id, score = line.split(' ')
    #     if not(passage_id in turn_passages_test[turn_id]):
    #         new_lines.append(line.replace(' ', '\t'))

    # assert len(new_lines)==len(lines)

    return new_lines

def without_filling():
    turn_passages_test = get_test_passage_query_pairs()
    new_lines = []


    for turn_id in turn_passages_test:
        for passage_id in  turn_passages_test[turn_id]:
            line = turn_id +'\t0\t'+ passage_id+'\t0\n' 
            new_lines.append(line)

    assert len(new_lines) == 917
    with open(nist_pool_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        turn_id, _, passage_id, score = line.split(' ')
        if not(passage_id in turn_passages_test[turn_id]):
            new_lines.append(line.replace(' ', '\t'))

    assert len(new_lines)==len(lines)

    with open('tmp_pools/no_filling.txt', 'w') as f:
        f.writelines(new_lines)
    
    return

without_filling()

{'train', 'NONE', 'validation', 'test'}
917


In [ ]:
nist_pool_path = 'data/human_qrels'


model_names = [file_name for file_name in os.listdir('outputs/') if file_name.endswith('.txt')]
for model_name in model_names:
    gpt_pool_path = 'outputs/'+model_name
    partial_pool_path = 'tmp_pools/'+model_name

    new_tmp_pool = create_15_percent_pool(gpt_pool_path, nist_pool_path)
    with open(partial_pool_path, 'w') as f:
        f.writelines(new_tmp_pool)

In [ ]:
model_name = 'human_qrels_tab.txt'
gpt_pool_path = 'outputs/'+model_name
partial_pool_path = 'tmp_pools/'+model_name

new_tmp_pool = create_15_percent_pool(gpt_pool_path, nist_pool_path)
with open(partial_pool_path, 'w') as f:
    f.writelines(new_tmp_pool)